In [1]:
import numpy as np
import cv2
import math

In [2]:
def separateColors(image):
    height, width, _ = image.shape
    blanco = np.zeros((height,width,1), np.uint8)
    first = np.zeros((height,width,1), np.uint8)
    last = np.zeros((height,width,1), np.uint8)
    for x in range(height):
        for y in range(width):
            if(image[x][y][0] == 255 and image[x][y][1]== 255 and image[x][y][2]== 255):
                blanco[x][y][0] = 255
                first[x][y][0] = 255
                last[x][y][0] = 255
            if(image[x][y][0] == 255):
                last[x][y][0] = 255
            if(image[x][y][2]== 255):
                first[x][y][0] = 255
    return (blanco, first, last)

In [3]:
def detectStationary(img, c):
    conflicto = False
    i=0
    color = [255, 255, 255]
    while(conflicto == False and i < len(c)):
        conflicto, newColor = whiteSurroundings(img, c[i][0])
        if(not(np.array_equal(newColor, color)) and not(np.array_equal(newColor,[255, 255, 255])) and not(np.array_equal(newColor,[0, 0, 0]))):
            if(np.array_equal(color,[255, 255, 255])):
                color = newColor
            else:
                conflicto = True
        i=i+1
    return not conflicto

In [4]:
def whiteSurroundings(img, pto):
    conflict = False
    color = [255, 255, 255]
    rows, cols,_ = image.shape
    i = -1
    while(i<2 and conflict == False):
        j=-1
        while(j<2 and conflict == False):
            if(pto[1]+ i >=1 and pto[1] +i < rows and pto[0]+ j >=1 and pto[0]+j < cols):
                if (not((img[pto[1]+i][pto[0]+j] ==[255, 255, 255]).all() or (img[pto[1]+i][pto[0]+j] ==[0, 0, 0]).all())):
                    if(np.array_equal(color,[255, 255, 255])):
                        color =img[pto[1]+i][pto[0]+j]
                    elif(not(np.array_equal(color,img[pto[1]+i][pto[0]+j]))):
                        conflict=True
            j=j+1
        i=i+1
    return conflict, color

In [5]:
def getUseless(image, white):
    cntsWhite= cv2.findContours(white, cv2.RETR_LIST ,cv2.CHAIN_APPROX_NONE )[1]
    listUseless=[]
    for c in cntsWhite:
        if (cv2.contourArea(c)>0):
            if(detectStationary(image, c)):
                listUseless.append(c)
    return listUseless

In [6]:
def inContourUseless(c, listCont):
    for i in listCont:
        for pto in i:
            if(cv2.pointPolygonTest(c, (pto[0][0],pto[0][1]), False)!=-1):
                return True
    return False

In [7]:
def getCenters(cnts, listUseless):
    listCenter =[]
    for c in cnts:
        if(cv2.contourArea(c)>0 and not(inContourUseless(c, listUseless))):
            M = cv2.moments(c)
            cntX = int(M["m10"] / M["m00"])
            cntY = int(M["m01"] / M["m00"])
            listCenter.append((cntX,cntY))
    return listCenter

In [8]:
def matrixCenter(centerFirst, centerLast):
    matrix = [[(0,0) for x in range(len(centerLast))] for y in range(len(centerFirst))]
    for i in range(len(centerFirst)):
        for j in range(len(centerLast)):
            dist=math.sqrt((centerFirst[i][0]-centerLast[j][0])**2+(centerFirst[i][1]-centerLast[j][1])**2)
            ang=math.atan2(centerFirst[i][0]-centerLast[j][0], centerFirst[i][1]-centerLast[j][1]) * (180.0 / math.pi)
            matrix[i][j] =(dist,ang)
    return matrix

In [9]:
image = cv2.imread('imagen.png', 1)
white, first, last = separateColors(image)

cntsFirst= cv2.findContours(first, cv2.RETR_LIST ,cv2.CHAIN_APPROX_NONE )[1]
cntsLast= cv2.findContours(last, cv2.RETR_LIST ,cv2.CHAIN_APPROX_NONE )[1]

listUseless = getUseless(image, white)
centerLast = getCenters(cntsLast, listUseless)
centerFirst = getCenters(cntsFirst, listUseless)

In [10]:
matriz = matrixCenter(centerFirst,centerLast)
print(matriz)

[[(190.09471323527123, -91.80873932249206), (27.16615541441225, -83.6598082540901), (6.0, -90.0), (663.5276934687805, 86.11125401664012), (280.49420671379295, -64.89672483921319), (148.47558721890948, 10.477423596004869), (439.16397848639633, 63.493294810404635), (432.9214709390145, 63.967642167734084), (784.8649565371103, 66.17834124772975), (785.60040733187, 65.0007305691796), (458.9172038614373, 45.26484763372736), (769.8947980081434, 64.86621322305417), (459.91847103590004, 24.949159105921712), (1025.0170730285422, 63.584937431909474), (608.2474825266439, 27.19696429889896)], [(144.00347218036097, 153.6128854543324), (256.76643082770767, 117.86246898247626), (276.82666056577716, 116.37992813948502), (919.3149623496835, 94.86715698823892), (4.0, 180.0), (281.93970986719836, 85.32074083959517), (651.1052142319243, 83.56263097360593), (646.4812448942351, 84.05130048397665), (991.1710245966636, 78.71274096346386), (988.3506462789409, 77.79187427534922), (613.5144660071187, 70.974393962

In [11]:
for pto in centerLast:
    cv2.circle(image, pto, 2, (0, 255, 0), -1)
for pto in centerFirst:
    cv2.circle(image, pto, 2, (255, 175, 0), -1)

cv2.imshow("imagen", image)
cv2.waitKey(0)
cv2.destroyAllWindows()